## Faster RAG with Guardrails

> https://www.pinecone.io/learn/fast-retrieval-augmented-generation/

---

Retrieval Augmented Generation (RAG) allows us to update the knowledge of LLMs via the model's "source knowledge". The source knowledge refers to any information fed into the LLM via the input prompt.

![](https://cdn.sanity.io/images/vr8gru94/production/caa233388a98715de2036dbf2bf2743e6e049b36-2283x1145.png)


Using RAG and source knowledge, we retrieve relevant information from an external data source, augment our prompt with this additional source knowledge, and feed that information into the LLM. This approach is compelling because it allows us to keep LLM knowledge accurate and up to date with minimal cost.

The above describes what I refer to as "naive RAG". Naive RAG is the most common implementation. It is simple, efficient, and effective.

However, there are other ways of doing RAG. The other popular approach is to use agents that have access to RAG tools.

At its core, an agent is an LLM with added logic that allows it to reason over multiple generation steps, decide to use various tools, and specify how to use those tools.

![](https://cdn.sanity.io/images/vr8gru94/production/4defbaed3e9f223a658a87b9b625da131048c84d-2240x2024.png)

This methodology allows the agent to answer far more complex queries. Rather than needing to produce an answer in a single step, it can now formulate and execute a plan to construct the best possible answer.

RAG is one of the most common tools to give agents access to external knowledge. The advantage of this approach over the naive RAG approach primarily stems from the more advanced reasoning ability of the agent. It can decide __when to use RAG and how to use it__. An agent can formulate a better search query based on the user's question, its parametric knowledge, and conversational history — it can also decide on additional parameters like how many results to return or whether to use metadata filtering.



The problem with agent-powered RAG is speed and cost. 

- Every reasoning step taken by an agent is a call to an LLM — LLMs are slow and expensive — so we will be waiting longer for a response and paying more money.


These two approaches to RAG are the most common. We have naive RAG and agent RAG. However, a third option provides us with a middle ground between the two, a best of both. I like to call that option RAG with Guardrails.


### RAG with Guardrails

Rather than using a slow and expensive LLM call to decide which action to take, we can use guardrails. Guardrails can be described as classifiers of user intent. We can create a guardrail to identify queries that indicate someone is asking a question — when a user asks a question, the guardrails identify this intent and trigger the RAG pipeline.


![](https://cdn.sanity.io/images/vr8gru94/production/f4c238be0ec0baa83e51ff227d26e8f3df1b7c8b-2895x1710.png)


The categories of user intent are called "canonical forms". We define a canonical form by providing example queries (utterances) that we believe belong to that canonical form.

If we had a RAG pipeline that contained information about LLMs, we could define a canonical form that identifies when a user asks about LLMs. In the image above, we define this canonical form with `"define user ask llm"`. We then provide a few example queries that should trigger this form:


Each utterance is encoded into a semantic vector space, creating a semantic "map". That map helps us line up semantic meaning and canonical forms.

![](https://cdn.sanity.io/images/vr8gru94/production/f8c83210f38b76eb84a589f8231ce69d8565c112-2293x1534.png)

When a new user query comes in, we encode it into the same semantic vector space as our previous utterance examples. 

Using that map of utterances and the canonical forms they belong to, we can identify when a user query is semantically similar to those utterance examples. 
If the query is similar, we trigger the canonical form that those utterances belong to.

With that, we have a decision-making process in milliseconds rather than several seconds — as with agents. __This is because we only used embeddings and NOT an LLM to do this decision making__


Using RAG with guardrails does have nuances that we should consider. First, we must define the canonical forms. We could view the requirement of defining canonical forms in two ways: (1) we can be more deterministic about what should trigger a particular action, or (2) we lose out on the innate decision-making ability of an LLM.


RAG with guardrails allows us to insert a user query directly into an action — but it cannot rephrase the query, specify metadata filters, or decide how many results to return. On the other hand, using an agent makes those tasks easy. However, if we can infer these parameters deterministically with code, we can include them without using an LLM.

